In [1]:
index = 1
key = 263
patch_size = 64
stride = 1
h5f = 'train.h5'
hyper_name = 'data/BGU/train_spectral/BGU_HS_00263.mat'
rgb_name = 'data/BGU/train_clean/BGU_HS_00263_clean.png'
mode = 'train'

In [21]:
import os
import h5py
import numpy as np
import cv2
from scipy.interpolate import interp1d
from shutil import copyfile
import matplotlib.pyplot as plt
import scipy.io
from tqdm import tqdm

In [22]:
mathy =  h5py.File(hyper_name,'r')
hyper = np.float16(np.array(mathy['rad']))
hyper = np.transpose(hyper, [2,0,1])
hyper = normalize(hyper, max_val=4095., min_val=0.)

In [23]:
def normalize(data, max_val, min_val):
    return (data-min_val)/(max_val-min_val)

In [19]:
rgb =  cv2.imread(rgb_name)
rgb = cv2.cvtColor(rgb, cv2.COLOR_BGR2RGB)
rgb = np.transpose(rgb, [2,0,1])

In [24]:
real_rgb = rgb
zeros = np.ones([28, real_rgb.shape[1], real_rgb.shape[2]], dtype=np.float32)
real_rgb = np.append(real_rgb, zeros, axis=0)
x1 = np.linspace(0, 1, num=3)
x2 = np.linspace(0, 1, num=31)
for m in tqdm(range(real_rgb.shape[1])):
    for j in range(real_rgb.shape[2]):
        f = interp1d(x1, real_rgb[0:3,m,j])
        real_rgb[:,m,j] = f(x2)

100%|███████████████████████████████████████| 1392/1392 [03:56<00:00,  5.88it/s]


In [26]:
def Im2Patch(img, win, stride=1):
    k = 0
    endc = img.shape[0]
    endw = img.shape[1]
    endh = img.shape[2]
    patch = img[:, 0:endw-win+0+1:stride, 0:endh-win+0+1:stride]
    TotalPatNum = patch.shape[1] * patch.shape[2]
    Y = np.zeros([endc, win*win,TotalPatNum], np.float32)
    for i in tqdm(range(win)):
        for j in range(win):
            patch = img[:,i:endw-win+i+1:stride,j:endh-win+j+1:stride]
            Y[:,k,:] = np.array(patch[:]).reshape(endc, TotalPatNum)
            k = k + 1
    return Y.reshape([endc, win, win, TotalPatNum])

In [35]:
hyper = np.transpose(hyper, [1, 2, 0])

(31, 1392, 1300)

In [42]:
patches_hyper = Im2Patch(hyper, win=patch_size, stride=patch_size)

100%|███████████████████████████████████████████| 64/64 [00:00<00:00, 75.76it/s]


In [44]:
patches_rgb = Im2Patch(real_rgb, win=patch_size, stride=patch_size)

100%|███████████████████████████████████████████| 64/64 [00:01<00:00, 43.21it/s]


In [37]:
patches_hyper.shape

(31, 64, 64, 420)

In [46]:
patches_rgb.shape

(31, 64, 64, 420)

In [49]:
hf = h5py.File(h5f, 'w')
for j in range(patches_hyper.shape[3]):
    print("generate training sample {}".format(index))
    sub_hyper = patches_hyper[:,:,:,j]
    sub_rgb = patches_rgb[:,:,:,j]
    data = np.concatenate(( sub_rgb, sub_hyper), 0)
    hf.create_dataset(str(index), data=data)
    index += 1

generate training sample 1
generate training sample 2
generate training sample 3
generate training sample 4
generate training sample 5
generate training sample 6
generate training sample 7
generate training sample 8
generate training sample 9
generate training sample 10
generate training sample 11
generate training sample 12
generate training sample 13
generate training sample 14
generate training sample 15
generate training sample 16
generate training sample 17
generate training sample 18
generate training sample 19
generate training sample 20
generate training sample 21
generate training sample 22
generate training sample 23
generate training sample 24
generate training sample 25
generate training sample 26
generate training sample 27
generate training sample 28
generate training sample 29
generate training sample 30
generate training sample 31
generate training sample 32
generate training sample 33
generate training sample 34
generate training sample 35
generate training sample 36
g

In [50]:
index

421